# RAG

## 下載使用者文件

In [1]:
! wget https://kaas.hpcloud.hp.com/pdf-public/pdf_10252811_en-US-1.pdf

--2025-06-26 15:11:49--  https://kaas.hpcloud.hp.com/pdf-public/pdf_10252811_en-US-1.pdf
Resolving kaas.hpcloud.hp.com (kaas.hpcloud.hp.com)... 18.239.50.37, 18.239.50.34, 18.239.50.12, ...
Connecting to kaas.hpcloud.hp.com (kaas.hpcloud.hp.com)|18.239.50.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10406809 (9.9M) [application/pdf]
Saving to: ‘pdf_10252811_en-US-1.pdf’

pdf_10252811_en-US- 100%[===================>]   9.92M  13.3MB/s    in 0.7s    

2025-06-26 15:11:51 (13.3 MB/s) - ‘pdf_10252811_en-US-1.pdf’ saved [10406809/10406809]



## 安裝必要的套件

安裝LangChain、OpenAI、PyPDF2、chromadb等套件


In [3]:
%pip install langchain openai pypdf2 chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 12.6 MB/s eta

## 載入pdf文件

使用PyPDF2載入PDF文件


In [4]:
from PyPDF2 import PdfReader

with open('pdf_10252811_en-US-1.pdf', 'rb') as file:
    # 使用PdfReader類別讀取檔案
    pdf_reader = PdfReader(file)

    # 從PDF提取文本
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()


## 分割文本

將PDF文本分割成更小的Chunk以便處理


In [5]:
from langchain.text_splitter import CharacterTextSplitter

# chunk_size 設定每個Chunk的最大長度為 1000 個字元
# chunk_overlap 設定相鄰Chunk之間的重疊字元數為 200
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# 使用 CharacterTextSplitter 物件的 split_text 方法將之前載入的PDF文本 text 分割成Chunk
docs = text_splitter.split_text(text)

## 建立Embedding模型

初始化一個OpenAI嵌入模型


In [8]:
%pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
api_key = ''

In [9]:
from langchain_community.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=api_key)

/tmp/ipython-input-9-2406525955.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=api_key)


## 建立向量儲存

使用ChromaDB作為向量儲存並從Chunk建立Embedding


In [11]:
from langchain.schema import Document

# 將字串列表 docs 轉換為 Document 物件
docs_objects = [Document(page_content=d) for d in docs]

# 將建立好的向量儲存到 db 中
db = Chroma.from_documents(docs_objects, embeddings)

## 建立LangChain Retrieval Chain

設定LangChain Retrieval Chain，包括Retriever 還有 LLM


In [13]:
%pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.0 MB/s eta 0:00:00


In [14]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


# 將 temperature 參數設定為 0
llm = ChatOpenAI(openai_api_key=api_key, temperature=0)

# 從向量儲存實例 db 建立一個Retriever
retriever = db.as_retriever()

# 定義一個聊天提示範本
# 包含一個系統訊息和一個使用者訊息
# 系統訊息應指示模型根據提供的上下文回答問題
# 使用者訊息應包含一個問題
template = """根據以下上下文回答問題：
{context}

問題：{question}
"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{question}")])

# 建立一個 LangChain Retrieval Chain
# 將Retriever、提示範本和語言模型串聯起來
# Chain應使用 RunnableParallel 來平行處理上下文和問題
# 然後將結果傳遞給提示範本和語言模型
# 使用 RunnablePassthrough() 來傳遞原始問題
# 最後，使用 StrOutputParser() 來解析輸出
rag_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

## 執行rag查詢

使用Retrieval Chain執行RAG查詢並獲取答案


In [17]:
# 定義一個字串變數，例如 query，並賦予您想要詢問RAG系統的問題。
query = "如何使用密碼"

# 使用先前建立的 rag_chain 物件，調用其 .invoke() 方法，並將 query 變數作為參數傳入。
# 將 rag_chain.invoke(query) 的結果儲存到一個新的變數中，例如 response。
response = rag_chain.invoke(query)

# 打印或顯示 response 變數的內容，以查看RAG系統針對您的查詢所產生的答案。
print(response)

您可以使用密碼來保護您的電腦信息和在線交易。您可以設置多種類型的密碼，例如用戶密碼、Windows密碼或HP Setup Utility（BIOS）密碼。以下是一些使用密碼的提示：

1. 為了減少被鎖定在電腦外的風險，記錄每個密碼並將其存儲在遠離電腦的安全地方。不要將密碼存儲在電腦上的文件中。
2. 在創建密碼時，遵循程序設定的要求。
3. 每3個月至少更改一次密碼。
4. 理想的密碼應該是長的，包含字母、標點符號、符號和數字。
5. 在將電腦送修之前，備份您的文件，刪除機密文件，然後刪除所有密碼設置。

另外，您可以通過搜索HP支持助手應用程序或點擊任務欄中的問號圖標來獲取有關Windows密碼的額外信息。
